In [1]:
import os
from collections import deque
from dotenv import load_dotenv
from google import genai
import pandas as pd
from pydantic import BaseModel, Field
import time
from typing import Annotated, Literal

folder_path = 'data/italy/'

ieee_file = folder_path + 'ieee_italy.csv'
scopus_file = folder_path + 'scopus_italy.csv'
wos_file = folder_path + 'wos_italy.xls'

# IEEE Xplore

In [2]:
df_ieee = pd.read_csv(ieee_file)
df_ieee

,Document Title,Authors,Author Affiliations,Publication Title,Date Added To Xplore,Publication Year,Volume,Issue,Start Page,End Page,...,Mesh_Terms,Article Citation Count,Patent Citation Count,Reference Count,License,Online Date,Issue Date,Meeting Date,Publisher,Document Identifier
0,"Use of satellite-based phenometrics, geophysic...",A. Mayer; B. Ortuani; M. De Peppo; F. Nutini; ...,Department of Agricultural and Environmental S...,2022 IEEE Workshop on Metrology for Agricultur...,7 Dec 2022,2022,NaN,NaN,64,69,...,NaN,4.0,NaN,17,IEEE,7 Dec 2022,NaN,NaN,IEEE,IEEE Conferences
1,Simulating soil greenhouse gases emissions wit...,M. Gabbrielli; M. Botta; M. Perfetto; I. Volpi...,Department of Agricultural and Environmental S...,2023 IEEE International Workshop on Metrology ...,12 Feb 2024,2023,NaN,NaN,216,221,...,NaN,NaN,NaN,17,IEEE,12 Feb 2024,NaN,NaN,IEEE,IEEE Conferences
2,Future rainfall scenarios for the assessment o...,R. Padulano; G. F. C. Lama; G. Rianna; M. Sant...,Regional Models and geo-Hydrological Impacts D...,2020 IEEE International Workshop on Metrology ...,8 Dec 2020,2020,NaN,NaN,241,246,...,NaN,18.0,NaN,16,IEEE,8 Dec 2020,NaN,NaN,IEEE,IEEE Conferences
3,PRIMA MAGO project: open-source applications b...,L. Pouget; A. Serra; F. Nuñez; M. Sarrias; S. ...,"Dept. Critical infrastructure, CETAQUA, Barcel...",2023 IEEE International Workshop on Metrology ...,12 Feb 2024,2023,NaN,NaN,531,536,...,NaN,NaN,NaN,8,IEEE,12 Feb 2024,NaN,NaN,IEEE,IEEE Conferences
4,Crop mapping applications at scale: Using Goog...,G. Lemoine; O. Léo,"European Commission, Institute for Environment...",2015 IEEE International Geoscience and Remote ...,12 Nov 2015,2015,NaN,NaN,1496,1499,...,NaN,17.0,NaN,5,IEEE,12 Nov 2015,NaN,NaN,IEEE,IEEE Conferences
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,A Novel Resource-Constrained Insect Monitoring...,A. Kargar; M. P. Wilk; D. Zorbas; M. T. Gaffne...,"Tyndall National Institute, University College...",2022 IEEE 5th International Conference on Imag...,28 Feb 2023,2022,Five,NaN,1,6,...,NaN,11.0,NaN,21,IEEE,28 Feb 2023,NaN,NaN,IEEE,IEEE Conferences
186,Improving Model Accuracy for Imbalanced Image ...,V. Kocaman; O. M. Shir; T. Bäck,"LIACS, Leiden University, Leiden, The Netherla...",2020 25th International Conference on Pattern ...,5 May 2021,2021,NaN,NaN,10404,10411,...,NaN,3.0,NaN,35,IEEE,5 May 2021,NaN,NaN,IEEE,IEEE Conferences
187,From RGB to Spectrum for Natural Scenes via Ma...,Y. Jia; Y. Zheng; L. Gu; A. Subpa-Asa; A. Lam;...,"RWTH Aachen University, Germany; National Inst...",2017 IEEE International Conference on Computer...,25 Dec 2017,2017,NaN,NaN,4715,4723,...,NaN,85.0,NaN,29,IEEE,25 Dec 2017,NaN,NaN,IEEE,IEEE Conferences
188,Pixel-Wise Classification of Hyperspectral Ima...,M. A. Shafaey; F. Melgani; M. A. . -M. Salem; ...,Faculty of Computer and Information Technology...,IEEE Access,29 Dec 2022,2022,10,NaN,133174,133185,...,NaN,17.0,NaN,41,CCBYNCND,21 Dec 2022,NaN,NaN,IEEE,IEEE Journals


In [3]:
df_ieee = df_ieee[['DOI', 'Document Title', 'Authors', 'Publication Year', 'Abstract', 'Article Citation Count']]

df_ieee = df_ieee.rename(columns={
    'Document Title': 'Title',
    'Publication Year': 'Year',
    'Article Citation Count': 'Citations'
})

df_ieee

,DOI,Title,Authors,Year,Abstract,Citations
0,10.1109/MetroAgriFor55389.2022.9964911,"Use of satellite-based phenometrics, geophysic...",A. Mayer; B. Ortuani; M. De Peppo; F. Nutini; ...,2022,In the most recent agricultural seasons water ...,4.0
1,10.1109/MetroAgriFor58484.2023.10424150,Simulating soil greenhouse gases emissions wit...,M. Gabbrielli; M. Botta; M. Perfetto; I. Volpi...,2023,"The complex interactions that soil, weather co...",NaN
2,10.1109/MetroAgriFor50201.2020.9277599,Future rainfall scenarios for the assessment o...,R. Padulano; G. F. C. Lama; G. Rianna; M. Sant...,2020,This research aims at understanding and analyz...,18.0
3,10.1109/MetroAgriFor58484.2023.10424068,PRIMA MAGO project: open-source applications b...,L. Pouget; A. Serra; F. Nuñez; M. Sarrias; S. ...,2023,The Copernicus program has significantly incre...,NaN
4,10.1109/IGARSS.2015.7326063,Crop mapping applications at scale: Using Goog...,G. Lemoine; O. Léo,2015,The confluence of rapidly growing streams of “...,17.0
...,...,...,...,...,...,...
185,10.1109/IPAS55744.2022.10052895,A Novel Resource-Constrained Insect Monitoring...,A. Kargar; M. P. Wilk; D. Zorbas; M. T. Gaffne...,2022,Effective insect pest monitoring is a vital co...,11.0
186,10.1109/ICPR48806.2021.9412907,Improving Model Accuracy for Imbalanced Image ...,V. Kocaman; O. M. Shir; T. Bäck,2021,"Some real-world domains, such as Agriculture a...",3.0
187,10.1109/ICCV.2017.504,From RGB to Spectrum for Natural Scenes via Ma...,Y. Jia; Y. Zheng; L. Gu; A. Subpa-Asa; A. Lam;...,2017,Spectral analysis of natural scenes can provid...,85.0
188,10.1109/ACCESS.2022.3231579,Pixel-Wise Classification of Hyperspectral Ima...,M. A. Shafaey; F. Melgani; M. A. . -M. Salem; ...,2022,"Nowadays, remote sensing image analysis is nee...",17.0


In [4]:
df_ieee = df_ieee.dropna(subset=['DOI'])
df_ieee = df_ieee.dropna(subset=['Citations'])
df_ieee

,DOI,Title,Authors,Year,Abstract,Citations
0,10.1109/MetroAgriFor55389.2022.9964911,"Use of satellite-based phenometrics, geophysic...",A. Mayer; B. Ortuani; M. De Peppo; F. Nutini; ...,2022,In the most recent agricultural seasons water ...,4.0
2,10.1109/MetroAgriFor50201.2020.9277599,Future rainfall scenarios for the assessment o...,R. Padulano; G. F. C. Lama; G. Rianna; M. Sant...,2020,This research aims at understanding and analyz...,18.0
4,10.1109/IGARSS.2015.7326063,Crop mapping applications at scale: Using Goog...,G. Lemoine; O. Léo,2015,The confluence of rapidly growing streams of “...,17.0
5,10.1109/MetroAgriFor55389.2022.9964826,Measuring the USLE soil erodibility factor in ...,V. Bagarello; V. Ferro; V. Pampalone,2022,The Universal Soil Loss Equation (USLE) is sti...,2.0
6,10.1109/MetroAgriFor.2019.8909219,2D and 3D data fusion for crop monitoring in p...,L. Comba; A. Biglia; D. R. Aimonino; P. Barge;...,2019,Addressing the intrinsic variability within vi...,31.0
...,...,...,...,...,...,...
185,10.1109/IPAS55744.2022.10052895,A Novel Resource-Constrained Insect Monitoring...,A. Kargar; M. P. Wilk; D. Zorbas; M. T. Gaffne...,2022,Effective insect pest monitoring is a vital co...,11.0
186,10.1109/ICPR48806.2021.9412907,Improving Model Accuracy for Imbalanced Image ...,V. Kocaman; O. M. Shir; T. Bäck,2021,"Some real-world domains, such as Agriculture a...",3.0
187,10.1109/ICCV.2017.504,From RGB to Spectrum for Natural Scenes via Ma...,Y. Jia; Y. Zheng; L. Gu; A. Subpa-Asa; A. Lam;...,2017,Spectral analysis of natural scenes can provid...,85.0
188,10.1109/ACCESS.2022.3231579,Pixel-Wise Classification of Hyperspectral Ima...,M. A. Shafaey; F. Melgani; M. A. . -M. Salem; ...,2022,"Nowadays, remote sensing image analysis is nee...",17.0


# Scopus

In [5]:
df_scopus = pd.read_csv(scopus_file)
df_scopus

,Authors,Author full names,Author(s) ID,Title,Year,Source title,Volume,Issue,Art. No.,Page start,...,ISSN,ISBN,CODEN,PubMed ID,Language of Original Document,Document Type,Publication Stage,Open Access,Source,EID
0,"C., Fabri, Charlotte; S., Vermeulen, Sam; S., ...","Fabri, Charlotte (57914677000); Vermeulen, Sam...",57914677000; 58554129300; 16053856800; 5721153...,Crop diversification and the effect of weather...,2024,Journal of Agricultural Economics,75.0,3,NaN,955.0,...,14779552; 0021857X,NaN,NaN,NaN,English,Article,Final,NaN,Scopus,2-s2.0-85196299289
1,"M., Bravi, Marina; M.C., Bottero, Marta Carla;...","Bravi, Marina (6701609410); Bottero, Marta Car...",6701609410; 57200325285; 57220855243,An Application of the Life Satisfaction Approa...,2024,Journal of the Knowledge Economy,15.0,1,NaN,2988.0,...,18687873; 18687865,NaN,NaN,NaN,English,Article,Final,NaN,Scopus,2-s2.0-85150366567
2,"R., Esposti, Roberto","Esposti, Roberto (6602726796)",6602726796,Non-monetary motivations of the EU agri-enviro...,2024,Journal of Environmental Management,352.0,NaN,119992.0,NaN,...,03014797; 10958630,NaN,JEVMA,38194870.0,English,Article,Final,All Open Access; Green Final Open Access; Gree...,Scopus,2-s2.0-85182420692
3,"P., Sau, Paola; B., Arru, Brunella; F., Cisili...","Sau, Paola (57827741900); Arru, Brunella (2603...",57827741900; 26031251200; 57208406771; 5607387...,Economic and Environmental Sustainability Trad...,2023,Sustainability (Switzerland),15.0,24,16621.0,NaN,...,20711050,NaN,NaN,NaN,English,Article,Final,NaN,Scopus,2-s2.0-85185349349
4,"G., Vono, Gregorio; G., Falcone, Giacomo; M., ...","Vono, Gregorio (57206721540); Falcone, Giacomo...",57206721540; 56032662600; 55914731800; 5673081...,Life Cycle Inventory of the Italian citrus fru...,2023,Cleaner Environmental Systems,10.0,NaN,100131.0,NaN,...,26667894,NaN,NaN,NaN,English,Article,Final,All Open Access; Gold Open Access,Scopus,2-s2.0-85164235180
5,"C., Russo, Carlo; F., Di Marcantonio, Federica...","Russo, Carlo (35733017800); Di Marcantonio, Fe...",35733017800; 57194794823; 56366879300; 2438541...,Unfair trading practices and countervailing power,2023,Food Policy,119.0,NaN,102521.0,NaN,...,03069192,NaN,NaN,NaN,English,Article,Final,All Open Access; Hybrid Gold Open Access,Scopus,2-s2.0-85167979613
6,"A., Boggia, Antonio; F.F., Fagioli, Filippo Fi...","Boggia, Antonio (56411997000); Fagioli, Filipp...",56411997000; 57193958324; 36194191400; 7103107...,Using accounting dataset for agricultural sust...,2023,International Transactions in Operational Rese...,30.0,4,NaN,2071.0,...,14753995; 09696016,NaN,NaN,NaN,English,Article,Final,All Open Access; Hybrid Gold Open Access,Scopus,2-s2.0-85127601569
7,"F., Cisilino, Federica; A., Giampaolo, Antonio...","Cisilino, Federica (57208406771); Giampaolo, A...",57208406771; 57221045085; 57199861605; 5841987...,The Tuscany Integrated Supply Chain Projects 2...,2023,Land,12.0,6,1230.0,NaN,...,2073445X,NaN,NaN,NaN,English,Article,Final,All Open Access; Gold Open Access,Scopus,2-s2.0-85163885884
8,"C., Cardillo, Concetta; A., Di Fonzo, Antonell...","Cardillo, Concetta (55513524600); Di Fonzo, An...",55513524600; 56723901600; 57552705100,The Farm’s Orientation towards Sustainability:...,2023,Land,12.0,2,301.0,NaN,...,2073445X,NaN,NaN,NaN,English,Article,Final,All Open Access; Gold Open Access,Scopus,2-s2.0-85149230331
9,"N., Galluzzo, Nicola","Galluzzo, Nicola (55348163300)",55348163300,How does eliminating the use of pesticides aff...,2023,Bulgarian Journal of Agricultural Science,29.0,1,NaN,14.0,...,13100351,NaN,NaN,NaN,English,Article,Final,NaN,Scopus,2-s2.0-85148943081


In [6]:
df_scopus = df_scopus[['DOI', 'Title', 'Authors', 'Year', 'Abstract', 'Cited by']]

df_scopus = df_scopus.rename(columns={
    'Cited by': 'Citations'
})

df_scopus

,DOI,Title,Authors,Year,Abstract,Citations
0,10.1111/1477-9552.12610,Crop diversification and the effect of weather...,"C., Fabri, Charlotte; S., Vermeulen, Sam; S., ...",2024,Agriculture is vulnerable to extreme weather s...,4
1,10.1007/s13132-023-01150-x,An Application of the Life Satisfaction Approa...,"M., Bravi, Marina; M.C., Bottero, Marta Carla;...",2024,"Land systems combine land use and land cover, ...",3
2,10.1016/j.jenvman.2023.119992,Non-monetary motivations of the EU agri-enviro...,"R., Esposti, Roberto",2024,This paper investigates the non-monetary motiv...,6
3,10.3390/su152416621,Economic and Environmental Sustainability Trad...,"P., Sau, Paola; B., Arru, Brunella; F., Cisili...",2023,Agriculture and livestock farming significantl...,2
4,10.1016/j.cesys.2023.100131,Life Cycle Inventory of the Italian citrus fru...,"G., Vono, Gregorio; G., Falcone, Giacomo; M., ...",2023,In the application of Life Cycle Assessment (L...,5
5,10.1016/j.foodpol.2023.102521,Unfair trading practices and countervailing power,"C., Russo, Carlo; F., Di Marcantonio, Federica...",2023,This paper investigates whether countervailing...,5
6,10.1111/itor.13141,Using accounting dataset for agricultural sust...,"A., Boggia, Antonio; F.F., Fagioli, Filippo Fi...",2023,Sustainability has become a key issue for agri...,12
7,10.3390/land12061230,The Tuscany Integrated Supply Chain Projects 2...,"F., Cisilino, Federica; A., Giampaolo, Antonio...",2023,The Integrated Supply Chain Projects (ISCP) ar...,4
8,10.3390/land12020301,The Farm’s Orientation towards Sustainability:...,"C., Cardillo, Concetta; A., Di Fonzo, Antonell...",2023,The new Common Agricultural Policy (CAP) for t...,9
9,NaN,How does eliminating the use of pesticides aff...,"N., Galluzzo, Nicola",2023,Through specific legislation such as Regulatio...,2


In [7]:
df_scopus = df_scopus.dropna(subset=['DOI'])
df_scopus = df_scopus.loc[df_scopus['Citations'] > 0]
df_scopus

,DOI,Title,Authors,Year,Abstract,Citations
0,10.1111/1477-9552.12610,Crop diversification and the effect of weather...,"C., Fabri, Charlotte; S., Vermeulen, Sam; S., ...",2024,Agriculture is vulnerable to extreme weather s...,4
1,10.1007/s13132-023-01150-x,An Application of the Life Satisfaction Approa...,"M., Bravi, Marina; M.C., Bottero, Marta Carla;...",2024,"Land systems combine land use and land cover, ...",3
2,10.1016/j.jenvman.2023.119992,Non-monetary motivations of the EU agri-enviro...,"R., Esposti, Roberto",2024,This paper investigates the non-monetary motiv...,6
3,10.3390/su152416621,Economic and Environmental Sustainability Trad...,"P., Sau, Paola; B., Arru, Brunella; F., Cisili...",2023,Agriculture and livestock farming significantl...,2
4,10.1016/j.cesys.2023.100131,Life Cycle Inventory of the Italian citrus fru...,"G., Vono, Gregorio; G., Falcone, Giacomo; M., ...",2023,In the application of Life Cycle Assessment (L...,5
5,10.1016/j.foodpol.2023.102521,Unfair trading practices and countervailing power,"C., Russo, Carlo; F., Di Marcantonio, Federica...",2023,This paper investigates whether countervailing...,5
6,10.1111/itor.13141,Using accounting dataset for agricultural sust...,"A., Boggia, Antonio; F.F., Fagioli, Filippo Fi...",2023,Sustainability has become a key issue for agri...,12
7,10.3390/land12061230,The Tuscany Integrated Supply Chain Projects 2...,"F., Cisilino, Federica; A., Giampaolo, Antonio...",2023,The Integrated Supply Chain Projects (ISCP) ar...,4
8,10.3390/land12020301,The Farm’s Orientation towards Sustainability:...,"C., Cardillo, Concetta; A., Di Fonzo, Antonell...",2023,The new Common Agricultural Policy (CAP) for t...,9
10,10.36253/rea-13209,The role of diversification in the revenue com...,"R., Henke, Roberto; O., Cimino, Orlando; F., V...",2022,The main aim of this paper is to analyse the e...,7


# Web of Science

In [8]:
df_wos = pd.read_excel(wos_file)
df_wos

,Publication Type,Authors,Book Authors,Group Authors,Book Group Authors,Researcher Ids,ORCIDs,Book Editors,Author - Arabic,Grant Principal Investigator,...,Copyright,Degree Name,Institution Address,Institution,Dissertation and Thesis Subjects,Author Keywords,Indexed Date,UT (Unique ID),Pubmed Id,Unnamed: 78
0,J,"Cogato, Alessia; Pezzuolo, Andrea; Sozzi, Marc...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2021-01-12,WOS:000600652300249,34026958.0,NaN
1,J,"Vizzari, Marco; Antognelli, Sara; Pauselli, Ma...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2016-12-07,WOS:000387906400003,NaN,NaN
2,J,"Notarnicola, B.; Tassielli, G.; Renzulli, P. A...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2022-01-13,WOS:000740394400001,NaN,NaN
3,J,"Quaresima, Sara; Nino, Pasquale; Cardillo, Con...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2025-01-03,WOS:001385694000001,NaN,NaN
4,J,"Pelosi, A.; Chirico, G. B.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2021-08-10,WOS:000706992800003,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,J,"Rondelli, Valda; Casazza, Camilla; Martelli, R...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-22,WOS:000455064700011,30553435.0,NaN
93,J,"Salata, Stefano",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2017-11-19,WOS:000413399800007,NaN,NaN
94,J,"Di Civita, Marika; Senczuk, Gabriele; Macciocc...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-12-13,WOS:001371948800001,NaN,NaN
95,J,"Bobbo, T.; Matera, R.; Biffani, S.; Gomez, M.;...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-02-28,WOS:001164002800001,37709038.0,NaN


In [9]:
df_wos = df_wos[['DOI', 'Article Title', 'Authors', 'Publication Year', 'Abstract', 'Times Cited, All Databases']]

df_wos = df_wos.rename(columns={
    'Article Title': 'Title',
    'Publication Year': 'Year',
    'Times Cited, All Databases': 'Citations'
})

df_wos

,DOI,Title,Authors,Year,Abstract,Citations
0,10.1016/j.dib.2020.106589,A sample of Italian vineyards: Landscape and m...,"Cogato, Alessia; Pezzuolo, Andrea; Sozzi, Marc...",2020,Agricultural land use plays a critical role in...,7
1,10.4018/IJAEIS.2016070102,Potential Nitrogen Load from Crop-Livestock Sy...,"Vizzari, Marco; Antognelli, Sara; Pauselli, Ma...",2016,The EU Water Directive establishes a common Eu...,5
2,10.1007/s11367-021-02020-x,Life cycle inventory data for the Italian agri...,"Notarnicola, B.; Tassielli, G.; Renzulli, P. A...",2024,Purpose For the development of any life cycle ...,24
3,10.3390/data9120149,Unlocking New Opportunities for Spatial Analys...,"Quaresima, Sara; Nino, Pasquale; Cardillo, Con...",2024,Italy is divided into 773 Agricultural Regions...,0
4,10.1016/j.agwat.2021.107169,Regional assessment of daily reference evapotr...,"Pelosi, A.; Chirico, G. B.",2021,This study evaluates the accuracy of daily ref...,60
...,...,...,...,...,...,...
92,10.1016/j.jsr.2018.09.015,"Tractor rollover fatalities, analyzing acciden...","Rondelli, Valda; Casazza, Camilla; Martelli, R...",2018,"Introduction: In many countries, traditional d...",54
93,10.1108/MEQ-07-2016-0049,Land use change analysis in the urban region o...,"Salata, Stefano",2017,Purpose - The comparative study between urban ...,23
94,10.1080/1828051X.2024.2432540,What conservation insights can we drag from ge...,"Di Civita, Marika; Senczuk, Gabriele; Macciocc...",2024,Italy is home to several indigenous goat breed...,0
95,10.3168/jds.2023-23629,Exploring the sources of variation of electric...,"Bobbo, T.; Matera, R.; Biffani, S.; Gomez, M.;...",2024,"In the buffalo dairy sector, a huge effort is ...",6


In [10]:
df_wos = df_wos.dropna(subset=['DOI', 'Abstract'])
df_wos = df_wos.loc[df_wos['Year'] != 2025]
df_wos = df_wos.loc[df_wos['Citations'] > 0]
df_wos

,DOI,Title,Authors,Year,Abstract,Citations
0,10.1016/j.dib.2020.106589,A sample of Italian vineyards: Landscape and m...,"Cogato, Alessia; Pezzuolo, Andrea; Sozzi, Marc...",2020,Agricultural land use plays a critical role in...,7
1,10.4018/IJAEIS.2016070102,Potential Nitrogen Load from Crop-Livestock Sy...,"Vizzari, Marco; Antognelli, Sara; Pauselli, Ma...",2016,The EU Water Directive establishes a common Eu...,5
2,10.1007/s11367-021-02020-x,Life cycle inventory data for the Italian agri...,"Notarnicola, B.; Tassielli, G.; Renzulli, P. A...",2024,Purpose For the development of any life cycle ...,24
4,10.1016/j.agwat.2021.107169,Regional assessment of daily reference evapotr...,"Pelosi, A.; Chirico, G. B.",2021,This study evaluates the accuracy of daily ref...,60
5,10.1016/j.agwat.2024.108732,Evaluation of daily crop reference evapotransp...,"Ippolito, Matteo; De Caro, Dario; Cannarozzo, ...",2024,Crop evapotranspiration (ET) is one of the mos...,21
...,...,...,...,...,...,...
91,10.1016/j.livsci.2015.06.013,Assessment of finishing beef cattle mortality ...,"Rumor, Clelia; Brscic, Marta; Contiero, Barbar...",2015,Cattle mortality has a relevant economic and e...,9
92,10.1016/j.jsr.2018.09.015,"Tractor rollover fatalities, analyzing acciden...","Rondelli, Valda; Casazza, Camilla; Martelli, R...",2018,"Introduction: In many countries, traditional d...",54
93,10.1108/MEQ-07-2016-0049,Land use change analysis in the urban region o...,"Salata, Stefano",2017,Purpose - The comparative study between urban ...,23
95,10.3168/jds.2023-23629,Exploring the sources of variation of electric...,"Bobbo, T.; Matera, R.; Biffani, S.; Gomez, M.;...",2024,"In the buffalo dairy sector, a huge effort is ...",6


# Final Dataset

In [11]:
df_all = pd.concat([df_ieee, df_scopus, df_wos], ignore_index=True)
df_all

,DOI,Title,Authors,Year,Abstract,Citations
0,10.1109/MetroAgriFor55389.2022.9964911,"Use of satellite-based phenometrics, geophysic...",A. Mayer; B. Ortuani; M. De Peppo; F. Nutini; ...,2022,In the most recent agricultural seasons water ...,4.0
1,10.1109/MetroAgriFor50201.2020.9277599,Future rainfall scenarios for the assessment o...,R. Padulano; G. F. C. Lama; G. Rianna; M. Sant...,2020,This research aims at understanding and analyz...,18.0
2,10.1109/IGARSS.2015.7326063,Crop mapping applications at scale: Using Goog...,G. Lemoine; O. Léo,2015,The confluence of rapidly growing streams of “...,17.0
3,10.1109/MetroAgriFor55389.2022.9964826,Measuring the USLE soil erodibility factor in ...,V. Bagarello; V. Ferro; V. Pampalone,2022,The Universal Soil Loss Equation (USLE) is sti...,2.0
4,10.1109/MetroAgriFor.2019.8909219,2D and 3D data fusion for crop monitoring in p...,L. Comba; A. Biglia; D. R. Aimonino; P. Barge;...,2019,Addressing the intrinsic variability within vi...,31.0
...,...,...,...,...,...,...
240,10.1016/j.livsci.2015.06.013,Assessment of finishing beef cattle mortality ...,"Rumor, Clelia; Brscic, Marta; Contiero, Barbar...",2015,Cattle mortality has a relevant economic and e...,9.0
241,10.1016/j.jsr.2018.09.015,"Tractor rollover fatalities, analyzing acciden...","Rondelli, Valda; Casazza, Camilla; Martelli, R...",2018,"Introduction: In many countries, traditional d...",54.0
242,10.1108/MEQ-07-2016-0049,Land use change analysis in the urban region o...,"Salata, Stefano",2017,Purpose - The comparative study between urban ...,23.0
243,10.3168/jds.2023-23629,Exploring the sources of variation of electric...,"Bobbo, T.; Matera, R.; Biffani, S.; Gomez, M.;...",2024,"In the buffalo dairy sector, a huge effort is ...",6.0


In [12]:
df_all = df_all.drop_duplicates(subset=['DOI'])
df_all = df_all.drop(columns=['Citations'])
df_all = df_all.reset_index(drop=True)
df_all

,DOI,Title,Authors,Year,Abstract
0,10.1109/MetroAgriFor55389.2022.9964911,"Use of satellite-based phenometrics, geophysic...",A. Mayer; B. Ortuani; M. De Peppo; F. Nutini; ...,2022,In the most recent agricultural seasons water ...
1,10.1109/MetroAgriFor50201.2020.9277599,Future rainfall scenarios for the assessment o...,R. Padulano; G. F. C. Lama; G. Rianna; M. Sant...,2020,This research aims at understanding and analyz...
2,10.1109/IGARSS.2015.7326063,Crop mapping applications at scale: Using Goog...,G. Lemoine; O. Léo,2015,The confluence of rapidly growing streams of “...
3,10.1109/MetroAgriFor55389.2022.9964826,Measuring the USLE soil erodibility factor in ...,V. Bagarello; V. Ferro; V. Pampalone,2022,The Universal Soil Loss Equation (USLE) is sti...
4,10.1109/MetroAgriFor.2019.8909219,2D and 3D data fusion for crop monitoring in p...,L. Comba; A. Biglia; D. R. Aimonino; P. Barge;...,2019,Addressing the intrinsic variability within vi...
...,...,...,...,...,...
220,10.1111/geoj.12153,The diffusion of grassroots innovations for su...,"Feola, Giuseppe; Butt, Anisa",2017,Little research so far has been devoted to und...
221,10.1016/j.livsci.2015.06.013,Assessment of finishing beef cattle mortality ...,"Rumor, Clelia; Brscic, Marta; Contiero, Barbar...",2015,Cattle mortality has a relevant economic and e...
222,10.1108/MEQ-07-2016-0049,Land use change analysis in the urban region o...,"Salata, Stefano",2017,Purpose - The comparative study between urban ...
223,10.3168/jds.2023-23629,Exploring the sources of variation of electric...,"Bobbo, T.; Matera, R.; Biffani, S.; Gomez, M.;...",2024,"In the buffalo dairy sector, a huge effort is ..."


# Abstract Evaluation with Gemini

In [13]:
load_dotenv()

MODEL= os.getenv("MODEL")
API_KEY = os.getenv("GEMINI_API_KEY3")

client = genai.Client(api_key=API_KEY)

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


In [14]:
class BatchReviewItem(BaseModel):
    id: int
    decision: Annotated[Literal["yes", "no", "maybe"], Field(
        description="Decision about including the paper:"
                    "- 'yes': There is clear evidence that the paper uses, shares, or discusses datasets, and it fully meets ALL inclusion criteria while NOT violating ANY exclusion criteria."
                    "- 'no': There is clear evidence that the paper should be excluded because it fails to meet all inclusion criteria or it meets at least one exclusion criterion."
                    "- 'maybe': The information is insufficient or ambiguous; checking the full text is recommended."
                    "If unsure, or it is needed to check other infos (like the full text) prefer 'maybe' over 'no'.")]
    reason: Annotated[str, Field(description="Short explanation for the decision")]
    data_source: Annotated[str, Field(
        description="Official and recognized name of the data source used in the paper if the dataset is explicitly mentioned."
                    "The dataset must be a real, identifiable resource (dataset, database, or data collection); check online if necessary."
                    "If only a vague or generic description is provided (e.g., 'agricultural data' or 'crop dataset' or 'generated data'), leave it empty."
                    "Use only the proper name of the resource, without adding descriptors about the type of data source (e.g., use 'PlantVillage' instead of 'PlantVillage dataset')."
                    "If multiple data sources are mentioned, insert all of them in the same string separated by ';' (semicolon)."
                    "If the decision is 'no', always leave it empty.")]


class BatchReview(BaseModel):
    items: list[BatchReviewItem]

fallback = BatchReviewItem(id=-1, decision="maybe", reason="No response", data_source="")


def is_open_data_relevant_batch(df_chunk: pd.DataFrame, client, model=None, max_retries=3):
    if model is None:
        model = MODEL if MODEL else "gemini-2.5-flash-lite"

    # Prepare record strings for prompt
    records = []
    for idx, row in df_chunk.iterrows():
        title = "" if pd.isna(row.get("Title")) else str(row.get("Title"))
        abstract = "" if pd.isna(row.get("Abstract")) else str(row.get("Abstract"))
        records.append(f"- id: {int(idx)}\n  Title: {title}\n  Abstract: {abstract}")

    # Build prompt with inclusion/exclusion criteria and instructions
    prompt = (
            "Evaluate each record based on the following Inclusion and Exclusion Criteria and return ONLY JSON that matches the provided schema.\n\n"
            "Inclusion Criteria:\n"
            "(a) The abstract mentions the use or production of datasets, databases, or other data sources.\n"
            "(b) The data sources are related to agriculture or closely related fields (e.g.: climate, soil...).\n"
            "(c) The data sources have exclusively Italian coverage (covering Italy at national level).\n\n"
            "Exclusion Criteria:\n"
            "(a) The abstract states that the data are not available or must be requested from the authors.\n"
            "(b) The data used or generated are not free (cost is required).\n"
            "(c) The data are global/regional.\n\n"
            "ALL the requirements should be met.\n\n"
            "Output Instructions:\n"
            "- For each provided record, return an object with: id, decision ('yes'|'no'|'maybe'), reason, data_source.\n"
            "- If title or abstract are missing or empty use decision='no' and reason='Title or abstract is missing', data_source=''.\n\n"
            "Records to evaluate:\n"
            + "\n".join(records)
    )

    global _sent_calls_count
    global _completed_evaluations_count
    backoff = 10

    # Error management with exponential backoff
    for _ in range(max_retries):
        _rate_limit_block_until_allowed()
        try:
            _sent_calls_count += 1
            resp = client.models.generate_content(
                model=model,
                contents=prompt,
                config={
                    "response_mime_type": "application/json",
                    "response_schema": BatchReview,
                    "temperature": 0.0
                },
            )
            # Update total evaluations count
            try:
                _completed_evaluations_count += len(df_chunk)
            except Exception as e:
                print(f"[attempt {_sent_calls_count}] Gemini error: {e}")
                _completed_evaluations_count = len(df_chunk)
            print(f"[GEMINI] Completed {_completed_evaluations_count} total evaluations.")
            return resp.parsed.items
        except Exception as e:
            print(f"[attempt {_sent_calls_count}] Gemini error: {e}")
            time.sleep(backoff)
            backoff *= 2
    return []


def _rate_limit_block_until_allowed():
    # Block until the number of calls in the window is below max
    while True:
        now = time.monotonic()
        while _rate_calls_log and (now - _rate_calls_log[0]) >= _RATE_WINDOW_SEC:
            _rate_calls_log.popleft()
        if len(_rate_calls_log) < _RATE_MAX_CALLS:
            _rate_calls_log.append(now)
            return


# Rate limiter: max 2 requests every 60 seconds
_RATE_WINDOW_SEC = 60.0
_RATE_MAX_CALLS = 2
_rate_calls_log = deque()

batch_size = 20  # number of papers per batch
df_evaluated = df_all.copy()

_sent_calls_count = 0
_completed_evaluations_count = 0

# Process dataframe in batches
for start in range(0, len(df_evaluated), batch_size):
    chunk = df_evaluated.iloc[start:start + batch_size]
    items = is_open_data_relevant_batch(chunk, client, model=MODEL)
    mapping = {int(it.id): it for it in items} if items else {}

    decisions = []
    reasons = []
    sources = []
    for idx in chunk.index:
        it = mapping.get(int(idx))
        if it is None:
            decisions.append(fallback.decision)
            reasons.append(fallback.reason)
            sources.append(fallback.data_source)
        else:
            decisions.append(it.decision)
            reasons.append(it.reason)
            sources.append(it.data_source)

    # Update dataframe with results
    df_evaluated.loc[chunk.index, 'Include'] = decisions
    df_evaluated.loc[chunk.index, 'Reason'] = reasons
    df_evaluated.loc[chunk.index, 'Data Source'] = sources

df_evaluated[['DOI', 'Include', 'Reason', 'Data Source']]

[GEMINI] Completed 20 total evaluations.
[GEMINI] Completed 40 total evaluations.
[GEMINI] Completed 60 total evaluations.
[GEMINI] Completed 80 total evaluations.
[GEMINI] Completed 100 total evaluations.
[GEMINI] Completed 120 total evaluations.
[GEMINI] Completed 140 total evaluations.
[GEMINI] Completed 160 total evaluations.
[GEMINI] Completed 180 total evaluations.
[GEMINI] Completed 200 total evaluations.
[GEMINI] Completed 220 total evaluations.
[GEMINI] Completed 225 total evaluations.


,DOI,Include,Reason,Data Source
0,10.1109/MetroAgriFor55389.2022.9964911,no,The study is conducted on a single farm in nor...,
1,10.1109/MetroAgriFor50201.2020.9277599,no,"The data source mentioned, EUROCORDEX, is a Eu...",
2,10.1109/IGARSS.2015.7326063,no,The abstract explicitly states the data source...,
3,10.1109/MetroAgriFor55389.2022.9964826,no,The data was collected at a specific experimen...,
4,10.1109/MetroAgriFor.2019.8909219,no,The abstract does not specify the geographical...,
...,...,...,...,...
220,10.1111/geoj.12153,no,"The study is not exclusively focused on Italy,...",
221,10.1016/j.livsci.2015.06.013,yes,The abstract mentions the use of the 'Italian ...,Italian National Cattle Movement database
222,10.1108/MEQ-07-2016-0049,yes,The paper uses national and regional land use ...,National Atlas
223,10.3168/jds.2023-23629,yes,The study uses a dataset on Italian Mediterran...,


In [15]:
df_evaluated.to_csv("data/italy/df_evaluated_italy.csv", index=False)

In [19]:
n_unique = df_evaluated['Data Source'].nunique()
n_unique

139

In [20]:
unique_sources = df_evaluated['Data Source'].unique().tolist()
unique_sources

['',
 'SMAP; NL-DAS; GLDAS; Global Data Assimilation System (GDAS); USGS land-cover',
 'ESA CCI-LC; JRC Land Productivity Dynamics (LPD); Landsat',
 'MODIS global land cover products; 30-m Global Land Cover Dataset',
 'MODIS; AIRS',
 'Advanced Microwave Scanning Radiometer—Earth Observing System; Global Land Data Assimilation System-Noah; ERA-Interim/Land; International Soil Moisture Network; ESA Climate Change Initiative—Soil Moisture (ESA CCI SM)',
 'ALOS-PALSAR; GlobeLand30',
 'MODIS',
 'Global Land Cover Data set; Landsat 5',
 'Tropical Ecology Assessment and Monitoring (TEAM) Network; Landsat; Gridded Population of the World (GPW); Global Roads Open Access Dataset (gROADS)',
 'Sentinel-2',
 'LandSat dataset; USGS Global SRTM dataset; global VCI dataset',
 'WorldClim',
 'GSMaP',
 'EPPO',
 'Sentinel',
 'LandCoverNet',
 'Global Satellite Mapping of Precipitation (GSMaP)',
 'Landsat; MODIS',
 'Black Marble',
 'Penman-Monteith-Leuning-version-2 (PMLv2); Global-dataset-of-solar-induced-

In [23]:
df_final = df_evaluated[df_evaluated["Data Source"] == ""]
df_final = df_final[df_final["Include"] != "no"]
df_final

,DOI,Title,Authors,Year,Abstract,Include,Reason,Data Source
30,10.1109/ICIRCA57980.2023.10220825,Sustainable Fertilizers in Coffee Plantation: ...,R. S; S. R,2023,"To improve soil fertility and crop yields, ear...",maybe,The abstract mentions the use of 'real-world d...,
33,10.1109/PICET60765.2024.10716047,Transforming Agriculture with Machine Learning...,D. P. Akurati; P. Devarahatti; S. K. Konakandl...,2024,Machine learning is a powerful tool for extrac...,maybe,The abstract mentions using 'actual datasets' ...,
34,10.1109/MetroAgriFor52389.2021.9628547,Live weight prediction of cattle using deep im...,A. Ruchay; K. Dorofeev; V. Kalschikov; V. Kolp...,2021,The traditional linear regression algorithm is...,maybe,The abstract mentions using 'real-world datase...,
36,10.1109/TAFE.2024.3447792,Real-Time Plant Disease Identification: Fusion...,P. S. Thakur; S. Chaturvedi; P. Khanna; T. She...,2024,"Climate change, adverse weather conditions, an...",maybe,The abstract mentions the use of five public d...,
39,10.1109/ICACS.2018.8333275,Evaluation of predictive data mining algorithm...,A. Arooj; M. Riaz; M. N. Akram,2018,Agricultural research has strengthened the opt...,maybe,The abstract mentions a 'dataset of agricultur...,
...,...,...,...,...,...,...,...,...
891,10.1016/j.dib.2024.110225,An agro-physiological dataset on industrial to...,"Carucci, Federica; Bregaglio, Simone; Caldarol...",2024,The availability of field experimental data pl...,maybe,No response,
892,10.3389/fsufs.2023.1216205,Global hotspots of climate change adaptation a...,"Costa Jr, Ciniro; Thornton, Philip; Wollenberg...",2023,As countries prioritize climate action under t...,maybe,No response,
894,10.1016/j.canrad.2023.06.016,Real world data in radiotherapy: A data farmin...,"Guihard, S.; Piot, M.; Issoufaly, I.; Giraud, ...",2023,The aim of the data farming project by the Uni...,maybe,No response,
897,10.1080/02626667.2021.1890749,A methodology to downscale water demand data w...,"Zogheib, Charles; Ochoa-Tocachi, Boris F.; Mou...",2021,Mountainous regions are a hotspot for water sc...,maybe,No response,
